## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
from ipywidgets.embed import embed_minimal_html
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Helong,CN,42.5397,128.9972,67.64,17,71,3.74,broken clouds
1,1,Kapaa,US,22.0752,-159.3190,73.40,78,75,12.66,broken clouds
2,2,Kefamenanu,ID,-9.4467,124.4781,82.65,45,32,5.39,scattered clouds
3,3,Busselton,AU,-33.6500,115.3333,78.01,49,99,1.99,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,79.95,76,77,19.13,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Kefamenanu,ID,-9.4467,124.4781,82.65,45,32,5.39,scattered clouds
3,3,Busselton,AU,-33.6500,115.3333,78.01,49,99,1.99,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,79.95,76,77,19.13,light rain
7,7,Oranjestad,AW,12.5240,-70.0270,79.00,83,20,14.97,few clouds
12,12,Saint-Philippe,RE,-21.3585,55.7679,80.60,65,0,16.11,light rain
15,15,Georgetown,MY,5.4112,100.3354,89.60,66,40,5.75,scattered clouds
29,29,Kavieng,PG,-2.5744,150.7967,84.13,67,92,4.32,overcast clouds
35,35,Qandala,SO,11.4720,49.8728,87.57,51,26,7.61,scattered clouds
37,37,Butaritari,KI,3.0707,172.7902,82.81,74,16,12.86,light rain
39,39,Banjar,ID,-8.1900,114.9675,87.80,74,20,18.41,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Kefamenanu,ID,82.65,scattered clouds,-9.4467,124.4781,
3,Busselton,AU,78.01,overcast clouds,-33.6500,115.3333,
4,Atuona,PF,79.95,light rain,-9.8000,-139.0333,
7,Oranjestad,AW,79.00,few clouds,12.5240,-70.0270,
12,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,
15,Georgetown,MY,89.60,scattered clouds,5.4112,100.3354,
29,Kavieng,PG,84.13,overcast clouds,-2.5744,150.7967,
35,Qandala,SO,87.57,scattered clouds,11.4720,49.8728,
37,Butaritari,KI,82.81,light rain,3.0707,172.7902,
39,Banjar,ID,87.80,few clouds,-8.1900,114.9675,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Kefamenanu,ID,82.65,scattered clouds,-9.4467,124.4781,Sokowindo Losmen
3,Busselton,AU,78.01,overcast clouds,-33.6500,115.3333,Observatory Guest House
4,Atuona,PF,79.95,light rain,-9.8000,-139.0333,Villa Enata
7,Oranjestad,AW,79.00,few clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
12,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
...,...,...,...,...,...,...,...
696,Eyl,SO,84.70,broken clouds,7.9803,49.8164,Amiin Hotel
697,Manggar,ID,86.76,few clouds,-2.8833,108.2667,Guest Hotel
700,Mecca,SA,87.57,clear sky,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
718,Gorontalo,ID,87.93,overcast clouds,0.5412,123.0595,Maqna Hotel by Prasanthi


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig
embed_minimal_html('export.html', views=[fig])